In [1]:
import sys
sys.path.append('..')

In [2]:
import src.dataset as dataset
from src.eigenface.pca import FacePCA

pca = FacePCA(gray=True)

In [3]:
pca.save_embedding('../dataset/eigenface_embeds.h5')
pca.to_onnx('../onnx/pca.onnx')

In [4]:
import onnxruntime as rt

sess = rt.InferenceSession("../onnx/pca.onnx", providers=["CPUExecutionProvider"])
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name

print(input_name, label_name)

face_features variable


In [5]:
import numpy as np
from sklearn.preprocessing import normalize

X = dataset.load(gray=True)
sample = X[0].astype(np.float32).reshape(1, -1)

pred = sess.run([label_name], {input_name: sample})[0]
print(pred.shape)
print(normalize(pred).shape)

(1, 384)
(1, 384)


In [6]:
import h5py
from numpy import linalg as LA

# load embedding data
h5f = h5py.File('../dataset/eigenface_embeds.h5','r')
train_embeds = np.copy(h5f['embeds'])
print(train_embeds[0].shape)
print(LA.norm(train_embeds[0]))
h5f.close()

(384,)
1.0


In [7]:

def cosine_similarity(A, B):
    return np.dot(A,B)/(LA.norm(A)* LA.norm(B))

a, b = train_embeds[0].reshape(-1), normalize(pred).reshape(-1)
print(a.shape, b.shape)
cosine_similarity(a, b)

(384,) (384,)


0.8825054476692247